# setup

In [2]:
import pandas as pd
import numpy as np
import ast

# draft

In [70]:
path = 'ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/'

In [71]:
Y = pd.read_csv(path + 'ptbxl_database.csv')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

In [72]:
AGG = pd.read_csv(path + 'scp_statements.csv', index_col = 0)
AGG = AGG[AGG.diagnostic == 1]

In [73]:
def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in AGG.index:
            tmp.append(AGG.loc[key].diagnostic_class)
    return list(set(tmp))

In [74]:
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

In [75]:
columns = ['exam_id', 'NORM', 'MI', 'STTC', 'CD', 'HYP']

In [79]:
def label2onehot(exam_id, labels):
    onehot = [np.nan, False, False, False, False, False]
    onehot[0] = exam_id
    for label in labels:
        if label == 'NORM':
            onehot[1] = True
        if label == 'MI':
            onehot[2] = True
        if label == 'STTC':
            onehot[3] = True
        if label == 'CD':
            onehot[4] = True
        if label == 'HYP':
            onehot[5] = True
    return onehot

In [80]:
data = []
for idx, row in Y.iterrows():
    exam_id = row['ecg_id']
    labels = row['diagnostic_superclass']
    # data.append([exam_id, labels])
    data.append(label2onehot(exam_id, labels))

In [82]:
metadata = pd.DataFrame(data, columns = columns)
metadata

,exam_id,NORM,MI,STTC,CD,HYP
0,1,True,False,False,False,False
1,2,True,False,False,False,False
2,3,True,False,False,False,False
3,4,True,False,False,False,False
4,5,True,False,False,False,False
...,...,...,...,...,...,...
21832,21833,False,False,True,False,False
21833,21834,True,False,False,False,False
21834,21835,False,False,True,False,False
21835,21836,True,False,False,False,False


In [83]:
metadata.to_csv('metadata.csv', index = None)